In [79]:
import torch

In [50]:
x1 = torch.tensor([0.0], requires_grad=True)
x2 = torch.tensor([0.3], requires_grad=True)
x3 = torch.tensor([0.4], requires_grad=True)

In [51]:
y1 = torch.tensor([0.1], requires_grad=True)
y2 = torch.tensor([-0.3], requires_grad=True)
y3 = torch.tensor([0.2], requires_grad=True)

In [61]:
x = torch.tensor([x1, x2, x3])
y = torch.tensor([y1, y2, y3])

In [71]:
r1 = torch.linalg.norm(x)
r2 = torch.linalg.norm(y)

In [63]:
r12 = torch.linalg.norm(x - y)

In [64]:
alpha_1 = torch.tensor([1.013], requires_grad=True)
alpha_2 = torch.tensor([0.2119], requires_grad=True)
alpha_3 = torch.tensor([0.1406], requires_grad=True)
alpha_4 = torch.tensor([0.003], requires_grad=True)

In [65]:
psi = torch.e ** (-2 * (r1 + r2)) * (1 + 0.5 * r12 * torch.e ** (-alpha_1 * r12)) * (1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2) ** 2 - alpha_4 * r12)

In [72]:
psi.backward()

In [69]:
first_derivative

(None, None, None, None, None, None)

In [68]:
second_derivative = torch.autograd.grad(first_derivative[0], x1)

TypeError: 'NoneType' object is not iterable

In [102]:
import autograd.numpy as np
from autograd import grad
from autograd import elementwise_grad as egrad

In [122]:
x = np.array([0.2, 0.3, 0.1])
y = np.array([0.1, 0.4, 0.3])
r1 = np.linalg.norm(x)
r2 = np.linalg.norm(y)
r12 = np.linalg.norm(x - y)
alpha_1 = 1.013
alpha_2 = 0.2119
alpha_3 = 0.1406
alpha_4 = 0.003

def psi(x, y, alpha_1, alpha_2, alpha_3, alpha_4):
    r1 = np.linalg.norm(x)
    r2 = np.linalg.norm(y)
    r12 = np.linalg.norm(x - y)

    return np.e ** (-2 * (r1 + r2)) * (1 + 0.5 * r12 * np.e ** (-alpha_1 * r12)) * (1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2) ** 2 - alpha_4 * r12)

In [123]:
import torch

# Define constants
alpha_1 = 1.013
alpha_2 = 0.2119
alpha_3 = 0.1406
alpha_4 = 0.003

# Define psi using torch
def psi(x, y):
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

# Convert numpy arrays to torch tensors
x = torch.tensor([0.2, 0.3, 0.1], dtype=torch.float64, requires_grad=True)
y = torch.tensor([0.1, 0.4, 0.3], dtype=torch.float64, requires_grad=True)

# Compute psi
psi_val = psi(x, y)

# Compute gradients (first derivatives)
grad_x, grad_y = torch.autograd.grad(psi_val, (x, y), create_graph=True)

# Compute second derivatives
# For example: second derivative of psi with respect to x[0] and x[1]
d2psi_dx0dx1 = torch.autograd.grad(grad_x[0], x, retain_graph=True)[0][1]  # ∂²ψ/∂x₀∂x₁

# You can construct the full Hessian for x or y like this:
def hessian(grad_vec, vars):
    return torch.stack([torch.autograd.grad(g, vars, retain_graph=True)[0] for g in grad_vec])

hess_x = hessian(grad_x, x)  # 3x3 Hessian wrt x
hess_y = hessian(grad_y, y)  # 3x3 Hessian wrt y


In [125]:
import torch

# Constants
alpha_1 = 1.013
alpha_2 = 0.2119
alpha_3 = 0.1406
alpha_4 = 0.003

# psi function using torch
def psi(x, y):
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

# Tensors with gradients enabled
x = torch.tensor([0.2, 0.3, 0.1], dtype=torch.float64, requires_grad=True)
y = torch.tensor([0.1, 0.4, 0.3], dtype=torch.float64, requires_grad=True)

# Compute psi
psi_val = psi(x, y)

# First gradients
grad_x = torch.autograd.grad(psi_val, x, create_graph=True)[0]
grad_y = torch.autograd.grad(psi_val, y, create_graph=True)[0]

# Laplacian with respect to x
laplacian_x = sum(torch.autograd.grad(grad_x[i], x, retain_graph=True)[0][i] for i in range(3))

# Laplacian with respect to y
laplacian_y = sum(torch.autograd.grad(grad_y[i], y, retain_graph=True)[0][i] for i in range(3))

print("Laplacian with respect to x:", laplacian_x.item())
print("Laplacian with respect to y:", laplacian_y.item())


Laplacian with respect to x: -0.5628513385054579
Laplacian with respect to y: -0.28447621380463595


In [128]:
import torch

# Define alphas as torch tensors with requires_grad=True
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True)
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)

alphas = [alpha_1, alpha_2, alpha_3, alpha_4]

# psi function using torch
def psi(x, y, alpha_1, alpha_2, alpha_3, alpha_4):
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

# Tensors with gradients enabled
x = torch.tensor([0.2, 0.3, 0.1], dtype=torch.float64, requires_grad=True)
y = torch.tensor([0.1, 0.4, 0.3], dtype=torch.float64, requires_grad=True)

# Compute psi
psi_val = psi(x, y, *alphas)

# First gradients w.r.t. x and y
grad_x = torch.autograd.grad(psi_val, x, create_graph=True)[0]
grad_y = torch.autograd.grad(psi_val, y, create_graph=True)[0]

# Laplacians
laplacian_x = sum(torch.autograd.grad(grad_x[i], x, retain_graph=True, create_graph=True)[0][i] for i in range(3))
laplacian_y = sum(torch.autograd.grad(grad_y[i], y, retain_graph=True, create_graph=True)[0][i] for i in range(3))

lap = laplacian_x + laplacian_y
# Derivatives of Laplacians w.r.t. alphas
dlap_dalpha = torch.autograd.grad(lap, alphas, retain_graph=True)


print(dlap_dalpha)

(tensor(-0.5851, dtype=torch.float64), tensor(2.5131, dtype=torch.float64), tensor(0.6771, dtype=torch.float64), tensor(-2.5998, dtype=torch.float64))


In [129]:
# Now I just need to add the PE and then we are golden